In [52]:
import pandas as pd
import numpy as np
import catboost
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from catboost import Pool
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.svm import SVR
from sklearn.linear_model import SGDRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import normalize
from sklearn import svm
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler

In [53]:
train = pd.read_csv('houses_to_rent_train.csv')
test = pd.read_csv('houses_to_rent_test.csv')
val = pd.read_csv('houses_to_rent_val.csv')

In [ ]:
#for catboost(to apply catboost without encoding of categorical features)
val2 = val.drop(['rent'], axis=1)
train2 = train.drop(['rent'], axis=1)
test2 = test.drop(['rent'],axis=1)

In [32]:
train.head()

,city,area,rooms,bathroom,parking spaces,floor,animal,furniture,Hassoc_tax,rent,prop_tax,fire_insur
0,São Paulo,64,2,1,0,8,acept,not furnished,500,1600,100,21
1,city1,98,3,3,2,0,acept,not furnished,0,2500,0,38
2,São Paulo,57,1,1,1,5,acept,not furnished,769,4800,194,61
3,São Paulo,182,3,2,1,6,acept,not furnished,1990,9000,283,115
4,São Paulo,100,2,1,0,0,not acept,not furnished,0,5000,125,76


In [33]:
test.head()

,city,area,rooms,bathroom,parking spaces,floor,animal,furniture,Hassoc_tax,rent,prop_tax,fire_insur
0,Campinas,103,3,2,1,9,not acept,not furnished,885,1800,113,23
1,São Paulo,180,3,5,3,3,acept,not furnished,2900,8100,0,103
2,Porto Alegre,62,3,2,1,1,not acept,not furnished,485,1629,54,24
3,São Paulo,250,2,2,1,0,acept,not furnished,0,4800,534,73
4,São Paulo,200,1,2,0,0,acept,not furnished,0,8000,700,121


In [34]:
train['city'].unique()

array(['São Paulo', 'city1', 'Campinas', 'Rio de Janeiro', 'Porto Alegre',
       'city0', 'Belo Horizonte'], dtype=object)

In [35]:
lev_enc = LabelEncoder()
train['animal']=lev_enc.fit_transform(train['animal'])
train['furniture']=lev_enc.fit_transform(train['furniture'])
val['animal']=lev_enc.fit_transform(val['animal'])
val['furniture']=lev_enc.fit_transform(val['furniture'])
test['animal']=lev_enc.fit_transform(test['animal'])
test['furniture']=lev_enc.fit_transform(test['furniture'])

In [36]:
train.head()

,city,area,rooms,bathroom,parking spaces,floor,animal,furniture,Hassoc_tax,rent,prop_tax,fire_insur
0,São Paulo,64,2,1,0,8,0,1,500,1600,100,21
1,city1,98,3,3,2,0,0,1,0,2500,0,38
2,São Paulo,57,1,1,1,5,0,1,769,4800,194,61
3,São Paulo,182,3,2,1,6,0,1,1990,9000,283,115
4,São Paulo,100,2,1,0,0,1,1,0,5000,125,76


In [37]:
#using one one_hot_encoder for 'city' col
hot_enc_train = pd.get_dummies(train['city'])
hot_enc_test = pd.get_dummies(test['city'])
hot_enc_val = pd.get_dummies(val['city'])

In [38]:
train_final = pd.concat([train, hot_enc_train], axis=1)
test_final = pd.concat([test, hot_enc_test], axis=1)
val_final = pd.concat([val, hot_enc_val], axis=1)

In [39]:
test_final.head()

,city,area,rooms,bathroom,parking spaces,floor,animal,furniture,Hassoc_tax,rent,prop_tax,fire_insur,Belo Horizonte,Campinas,Porto Alegre,Rio de Janeiro,São Paulo,city0,city1
0,Campinas,103,3,2,1,9,1,1,885,1800,113,23,0,1,0,0,0,0,0
1,São Paulo,180,3,5,3,3,0,1,2900,8100,0,103,0,0,0,0,1,0,0
2,Porto Alegre,62,3,2,1,1,1,1,485,1629,54,24,0,0,1,0,0,0,0
3,São Paulo,250,2,2,1,0,0,1,0,4800,534,73,0,0,0,0,1,0,0
4,São Paulo,200,1,2,0,0,0,1,0,8000,700,121,0,0,0,0,1,0,0


In [40]:
#Remove 'city' col from the data
X_train = train_final.drop(['city'], axis=1)
X_train.drop(['rent'], axis=1, inplace=True)
y_train = train_final['rent']
X_val =  val_final.drop(['city'], axis=1)
X_val.drop(['rent'], axis=1, inplace=True)
y_val = val_final['rent']
X_test =  test_final.drop(['city'], axis=1)
y_test = test_final['rent']

In [41]:
X_test.drop(['rent'], axis=1, inplace=True)

In [42]:
X_train.head()

,area,rooms,bathroom,parking spaces,floor,animal,furniture,Hassoc_tax,prop_tax,fire_insur,Belo Horizonte,Campinas,Porto Alegre,Rio de Janeiro,São Paulo,city0,city1
0,64,2,1,0,8,0,1,500,100,21,0,0,0,0,1,0,0
1,98,3,3,2,0,0,1,0,0,38,0,0,0,0,0,0,1
2,57,1,1,1,5,0,1,769,194,61,0,0,0,0,1,0,0
3,182,3,2,1,6,0,1,1990,283,115,0,0,0,0,1,0,0
4,100,2,1,0,0,1,1,0,125,76,0,0,0,0,1,0,0


In [43]:
y_train.head()

0    1600
1    2500
2    4800
3    9000
4    5000
Name: rent, dtype: int64

# CatBoost

In [55]:
train_pool = Pool(
                data = train2,
                label = y_train,
                cat_features = ['city', 'animal', 'furniture']
)
valid_pool = Pool(
                data = val2,
                label = y_val,
                cat_features = ['city', 'animal', 'furniture']
)

In [56]:
model = catboost.CatBoostRegressor(custom_metric= ['R2', 'RMSE'], learning_rate=0.1, n_estimators=2000)
model.fit(train_pool, eval_set=valid_pool, verbose=50, plot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 2088.0379925	test: 2127.7335290	best: 2127.7335290 (0)	total: 19.6ms	remaining: 39.2s
50:	learn: 264.3395012	test: 260.7071515	best: 260.7071515 (50)	total: 548ms	remaining: 20.9s
100:	learn: 236.7695031	test: 238.6984439	best: 238.6984439 (100)	total: 1.01s	remaining: 19s
150:	learn: 215.7853386	test: 230.6062864	best: 230.6062864 (150)	total: 1.52s	remaining: 18.6s
200:	learn: 202.7021820	test: 227.7120376	best: 227.7120376 (200)	total: 1.92s	remaining: 17.1s
250:	learn: 189.4964062	test: 225.6698492	best: 225.6698492 (250)	total: 2.4s	remaining: 16.7s
300:	learn: 178.7095504	test: 224.5721396	best: 224.5677945 (298)	total: 2.85s	remaining: 16.1s
350:	learn: 170.6379338	test: 222.5985437	best: 222.5085296 (345)	total: 3.31s	remaining: 15.5s
400:	learn: 163.5408154	test: 221.0374344	best: 221.0374344 (400)	total: 3.68s	remaining: 14.7s
450:	learn: 156.5560219	test: 221.3494086	best: 221.0286831 (401)	total: 3.99s	remaining: 13.7s
500:	learn: 150.1851561	test: 220.0670934	bes

In [59]:
y_hat = model.predict(test)

In [60]:
#error for test data
RMSE_cat = np.sqrt(mean_squared_error(y_test, y_hat))

In [61]:
RMSE_cat

182.89889118313585

In [62]:
r2score_cat = r2_score(y_test, y_hat) 

In [64]:
MAE = np.sum(np.absolute(y_test - y_hat))/y_hat.shape[0]

In [65]:
MAE

67.9959572835517

# SGR

Normalization

In [16]:
# fit scaler on training data
norm = MinMaxScaler().fit(X_train)

# transform training data
X_train_norm = norm.transform(X_train)

# transform testing dataabs
X_test_norm = norm.transform(X_test)

/home/nitinrana/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype uint8, int64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [23]:
X_train_norm = pd.DataFrame(X_train_norm)

In [24]:
X_train_norm.columns = X_train.columns

In [25]:
X_train_norm.head()

,area,rooms,bathroom,parking spaces,floor,animal,furniture,Hassoc_tax,prop_tax,fire_insur,Belo Horizonte,Campinas,Porto Alegre,Rio de Janeiro,São Paulo,city0,city1
0,0.164134,0.142857,0.000000,0.000,0.026578,0.0,1.0,0.000448,0.000273,0.101124,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,0.267477,0.285714,0.285714,0.250,0.000000,0.0,1.0,0.000000,0.000000,0.196629,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.142857,0.000000,0.000000,0.125,0.016611,0.0,1.0,0.000688,0.000530,0.325843,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0.522796,0.285714,0.142857,0.125,0.019934,0.0,1.0,0.001782,0.000773,0.629213,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0.273556,0.142857,0.000000,0.000,0.000000,1.0,1.0,0.000000,0.000341,0.410112,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [26]:
X_test_norm = pd.DataFrame(X_test_norm)
X_test_norm.columns = X_test.columns

Model building

In [32]:
reg = SGDRegressor(max_iter=2000, tol=1e-3)
reg.fit(X_train_norm, y_train)
pred = reg.predict(X_test_norm)

In [34]:
MAE1 = np.sum(np.absolute(y_test-pred))/pred.shape[0]

In [35]:
#MAE without tunning hyperparameters
MAE1

244.2195870003554

Tunning Hyperparameters with GridSearchCV

In [91]:
model = SGDRegressor(tol=1e-3, max_iter=2000, verbose=2,random_state=0)

In [92]:
param_grid = {
    'alpha': 10.0 ** -np.arange(1, 7),
    'loss': ['squared_loss', 'huber', 'epsilon_insensitive'],
    'penalty': ['l2', 'l1', 'elasticnet'],
    'learning_rate': ['constant', 'optimal', 'invscaling'],
}
clf = GridSearchCV(model, param_grid)
clf.fit(X_train, y_train)
print("Best score: " + str(clf.best_score_))

/home/nitinrana/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


-- Epoch 1
Norm: 811066704139878.50, NNZs: 17, Bias: 5080239293940.416016, T: 7665, Avg. loss: 2694733074476115349338472328571387904.000000
Total training time: 0.00 seconds.
-- Epoch 2
Norm: 17360886151538.95, NNZs: 17, Bias: -1667885747737.957520, T: 15330, Avg. loss: 3876970704206514218817429748188184576.000000
Total training time: 0.00 seconds.
-- Epoch 3
Norm: 12983731150886.11, NNZs: 17, Bias: 149718427815.402344, T: 22995, Avg. loss: 25177800900994762242562822299900182528.000000
Total training time: 0.00 seconds.
-- Epoch 4
Norm: 1428601186689575.25, NNZs: 17, Bias: -5167038924106.441406, T: 30660, Avg. loss: 2924780664557696107306796728310562816.000000
Total training time: 0.01 seconds.
-- Epoch 5
Norm: 14523198763042.48, NNZs: 17, Bias: -3382868335368.453125, T: 38325, Avg. loss: 8941172827110515830262413803535728640.000000
Total training time: 0.01 seconds.
-- Epoch 6
Norm: 10215771243111948.00, NNZs: 17, Bias: -3802184430637.409668, T: 45990, Avg. loss: 115453319796686573431

Norm: 6608825307003.85, NNZs: 17, Bias: 497693987389.723267, T: 38325, Avg. loss: 425463924574937264891086654930944.000000
Total training time: 0.01 seconds.
-- Epoch 6
Norm: 11879322869142.92, NNZs: 17, Bias: 642054782491.973755, T: 45990, Avg. loss: 338898077251637082306758118998016.000000
Total training time: 0.02 seconds.
-- Epoch 7
Norm: 17260522208507.78, NNZs: 17, Bias: 986110710206.483154, T: 53655, Avg. loss: 302078947470934786040123097088000.000000
Total training time: 0.02 seconds.
-- Epoch 8
Norm: 9175745918244.45, NNZs: 17, Bias: 963947605777.346191, T: 61320, Avg. loss: 281636792458373081957193427714048.000000
Total training time: 0.02 seconds.
Convergence after 8 epochs took 0.02 seconds
-- Epoch 1
Norm: 339471021464760.00, NNZs: 17, Bias: 2763997060562.202148, T: 7666, Avg. loss: 3762826464184025954190829282828222464.000000
Total training time: 0.00 seconds.
-- Epoch 2
Norm: 78642215122840.16, NNZs: 17, Bias: -3636002939437.797852, T: 15332, Avg. loss: 37397947066323241

-- Epoch 1
Norm: 220.01, NNZs: 4, Bias: 0.420000, T: 7666, Avg. loss: 9462.592629
Total training time: 0.00 seconds.
-- Epoch 2
Norm: 347.97, NNZs: 4, Bias: -0.142000, T: 15332, Avg. loss: 9258.660717
Total training time: 0.00 seconds.
-- Epoch 3
Norm: 488.44, NNZs: 1, Bias: 1.908000, T: 22998, Avg. loss: 9057.287625
Total training time: 0.01 seconds.
-- Epoch 4
Norm: 1077.87, NNZs: 2, Bias: 1.951993, T: 30664, Avg. loss: 3750.539108
Total training time: 0.01 seconds.
-- Epoch 5
Norm: 802.19, NNZs: 1, Bias: 2.101993, T: 38330, Avg. loss: 13139.800826
Total training time: 0.01 seconds.
-- Epoch 6
Norm: 864.19, NNZs: 3, Bias: 2.154116, T: 45996, Avg. loss: 9059.330355
Total training time: 0.02 seconds.
-- Epoch 7
Norm: 1039.75, NNZs: 2, Bias: 3.218116, T: 53662, Avg. loss: 7604.737093
Total training time: 0.02 seconds.
-- Epoch 8
Norm: 1170.09, NNZs: 3, Bias: 2.872116, T: 61328, Avg. loss: 7832.106619
Total training time: 0.02 seconds.
-- Epoch 9
Norm: 1333.54, NNZs: 3, Bias: 4.260116, T

-- Epoch 1
Norm: 2434.79, NNZs: 8, Bias: 2.330000, T: 7665, Avg. loss: 936084.556844
Total training time: 0.00 seconds.
-- Epoch 2
Norm: 1122.56, NNZs: 3, Bias: -0.300000, T: 15330, Avg. loss: 1004808.406029
Total training time: 0.01 seconds.
-- Epoch 3
Norm: 1649.32, NNZs: 0, Bias: -6.590000, T: 22995, Avg. loss: 964790.089760
Total training time: 0.01 seconds.
-- Epoch 4
Norm: 3530.84, NNZs: 5, Bias: 3.780000, T: 30660, Avg. loss: 911087.113629
Total training time: 0.01 seconds.
-- Epoch 5
Norm: 2699.84, NNZs: 3, Bias: 2.830000, T: 38325, Avg. loss: 1008036.795550
Total training time: 0.02 seconds.
-- Epoch 6
Norm: 11185.90, NNZs: 2, Bias: 5.460000, T: 45990, Avg. loss: 205900.844865
Total training time: 0.02 seconds.
-- Epoch 7
Norm: 3778.14, NNZs: 5, Bias: -3.550000, T: 53655, Avg. loss: 1697072.174599
Total training time: 0.02 seconds.
-- Epoch 8
Norm: 4139.54, NNZs: 4, Bias: 5.220000, T: 61320, Avg. loss: 956422.110680
Total training time: 0.03 seconds.
-- Epoch 9
Norm: 4689.68, 

...

-- Epoch 1
Norm: 248.71, NNZs: 17, Bias: 2.049914, T: 7665, Avg. loss: 121320.074984
Total training time: 0.00 seconds.
-- Epoch 2
Norm: 60.78, NNZs: 17, Bias: 1.757970, T: 15330, Avg. loss: 104224.192243
Total training time: 0.00 seconds.
-- Epoch 3
Norm: 60.50, NNZs: 17, Bias: 1.410059, T: 22995, Avg. loss: 83771.975429
Total training time: 0.01 seconds.
-- Epoch 4
Norm: 227.04, NNZs: 17, Bias: 2.255635, T: 30660, Avg. loss: 74342.823225
Total training time: 0.01 seconds.
-- Epoch 5
Norm: 66.98, NNZs: 17, Bias: 2.162125, T: 38325, Avg. loss: 75957.405443
Total training time: 0.01 seconds.
-- Epoch 6
Norm: 738.78, NNZs: 17, Bias: 2.252446, T: 45990, Avg. loss: 15166.162597
Total training time: 0.01 seconds.
-- Epoch 7
Norm: 95.89, NNZs: 17, Bias: 1.450472, T: 53655, Avg. loss: 115672.391534
Total training time: 0.02 seconds.
-- Epoch 8
Norm: 86.84, NNZs: 17, Bias: 2.021080, T: 61320, Avg. loss: 63628.190777
Total training time: 0.02 seconds.
-- Epoch 9
Norm: 92.60, NNZs: 17, Bias: 2.1

Norm: 74.67, NNZs: 17, Bias: -0.295616, T: 697515, Avg. loss: 448.366046
Total training time: 0.17 seconds.
-- Epoch 92
Norm: 74.28, NNZs: 17, Bias: -0.317005, T: 705180, Avg. loss: 444.447023
Total training time: 0.17 seconds.
-- Epoch 93
Norm: 74.49, NNZs: 17, Bias: -0.307668, T: 712845, Avg. loss: 442.894944
Total training time: 0.18 seconds.
-- Epoch 94
Norm: 74.67, NNZs: 17, Bias: -0.308380, T: 720510, Avg. loss: 443.238260
Total training time: 0.18 seconds.
-- Epoch 95
Norm: 74.14, NNZs: 17, Bias: -0.322755, T: 728175, Avg. loss: 439.111736
Total training time: 0.18 seconds.
-- Epoch 96
Norm: 75.62, NNZs: 17, Bias: -0.307357, T: 735840, Avg. loss: 444.398532
Total training time: 0.18 seconds.
-- Epoch 97
Norm: 74.73, NNZs: 17, Bias: -0.338053, T: 743505, Avg. loss: 441.485288
Total training time: 0.18 seconds.
-- Epoch 98
Norm: 74.88, NNZs: 17, Bias: -0.341114, T: 751170, Avg. loss: 442.249867
Total training time: 0.18 seconds.
-- Epoch 99
Norm: 74.85, NNZs: 17, Bias: -0.356389, 

Norm: 71.88, NNZs: 17, Bias: 0.415460, T: 260610, Avg. loss: 534.652284
Total training time: 0.16 seconds.
-- Epoch 35
Norm: 72.99, NNZs: 17, Bias: 0.440106, T: 268275, Avg. loss: 535.898665
Total training time: 0.16 seconds.
-- Epoch 36
Norm: 72.45, NNZs: 17, Bias: 0.419439, T: 275940, Avg. loss: 528.682848
Total training time: 0.17 seconds.
-- Epoch 37
Norm: 72.03, NNZs: 17, Bias: 0.390279, T: 283605, Avg. loss: 520.271623
Total training time: 0.17 seconds.
-- Epoch 38
Norm: 71.87, NNZs: 17, Bias: 0.385092, T: 291270, Avg. loss: 517.370285
Total training time: 0.18 seconds.
-- Epoch 39
Norm: 71.77, NNZs: 17, Bias: 0.378617, T: 298935, Avg. loss: 517.851005
Total training time: 0.18 seconds.
-- Epoch 40
Norm: 71.92, NNZs: 17, Bias: 0.363707, T: 306600, Avg. loss: 517.938790
Total training time: 0.18 seconds.
-- Epoch 41
Norm: 72.81, NNZs: 17, Bias: 0.370032, T: 314265, Avg. loss: 510.834398
Total training time: 0.19 seconds.
-- Epoch 42
Norm: 72.25, NNZs: 17, Bias: 0.344793, T: 321930

/home/nitinrana/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


-- Epoch 10
Norm: 1497.78, NNZs: 6, Bias: 4.497552, T: 114980, Avg. loss: 6471.652201
Total training time: 0.05 seconds.
-- Epoch 11
Norm: 1509.28, NNZs: 6, Bias: 3.740955, T: 126478, Avg. loss: 5467.513474
Total training time: 0.05 seconds.
-- Epoch 12
Norm: 1517.47, NNZs: 4, Bias: 3.081971, T: 137976, Avg. loss: 4783.861624
Total training time: 0.06 seconds.
-- Epoch 13
Norm: 1527.77, NNZs: 4, Bias: 3.381038, T: 149474, Avg. loss: 4392.349892
Total training time: 0.06 seconds.
-- Epoch 14
Norm: 1530.74, NNZs: 5, Bias: 3.052705, T: 160972, Avg. loss: 4244.394293
Total training time: 0.07 seconds.
-- Epoch 15
Norm: 1556.80, NNZs: 6, Bias: 3.098965, T: 172470, Avg. loss: 3262.458082
Total training time: 0.07 seconds.
-- Epoch 16
Norm: 1547.08, NNZs: 5, Bias: 3.579077, T: 183968, Avg. loss: 3852.757365
Total training time: 0.08 seconds.
-- Epoch 17
Norm: 1550.12, NNZs: 6, Bias: 3.180139, T: 195466, Avg. loss: 3431.576343
Total training time: 0.08 seconds.
-- Epoch 18
Norm: 1557.31, NNZs:

In [96]:
clf.get_params

<bound method BaseEstimator.get_params of GridSearchCV(cv='warn', error_score='raise-deprecating',
       estimator=SGDRegressor(alpha=0.0001, average=False, early_stopping=False, epsilon=0.1,
       eta0=0.01, fit_intercept=True, l1_ratio=0.15,
       learning_rate='invscaling', loss='squared_loss', max_iter=2000,
       n_iter=None, n_iter_no_change=5, penalty='l2', power_t=0.25,
       random_state=0, shuffle=True, tol=0.001, validation_fraction=0.1,
       verbose=2, warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'alpha': array([1.e-01, 1.e-02, 1.e-03, 1.e-04, 1.e-05, 1.e-06]), 'loss': ['squared_loss', 'huber', 'epsilon_insensitive'], 'penalty': ['l2', 'l1', 'elasticnet'], 'learning_rate': ['constant', 'optimal', 'invscaling']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)>

In [93]:
pred2 = clf.predict(X_test)

In [94]:
MAE2 = np.sum(np.absolute(y_test-pred2))/pred2.shape[0]

In [97]:
#MAE with hyperparameter tuning
MAE2 

230.73671769919216